In [1]:
import sys
import random
from itertools import product, chain
from collections import defaultdict

import tqdm
from datasets import Dataset

sys.path.insert(0, '/home/ogalolu/thesis/pre-training-multilingual-document-encoders/clef')
from evaluate import  _get_rerank_dir
from clef_dataloaders.clef_dataloader import load_clef_rerank, load_relevance_assessments, load_clef

# Config

In [2]:
qlang = "en"
dlang = "en"
root_dir = "/work/ogalolu/datasets"
year = "2002"
num_negative = 8

In [3]:
doc_ids, documents, query_ids, queries, relass = load_clef(qlang, dlang, year)
print("The number total documents:", len(documents))

The number total documents: 169477


# Initial Checks

In [4]:
s1 = set(relass.keys())
s2 = set(query_ids)
print("There are no relevance assesment for the following queries:", s2 - s1)

There are no relevance assesment for the following queries: {96, 132, 101, 110, 117, 118, 93, 127}


In [5]:
ids = list(chain.from_iterable(list(relass.values())))
print("Number of available query-document pairs:", len(ids))

Number of available query-document pairs: 821


# Obtain Training Instances

In [9]:
def find_negative(d_ind_list: list, num_negative: int) -> tuple:    
    """ Find negative examples by excluding the given list of document indices."""
    all_set = set(range(len(documents)))
    # To ensure that relevant documents won't appear in the set of negative examples
    available_set = all_set - set(d_ind_list)
    neg_ind_list = random.choices(tuple(available_set), k=num_negative)  
    assert not any(x in neg_ind_list for x in d_ind_list)
    negative_tuple = [documents[idx] for idx in neg_ind_list]
    
    return negative_tuple

In [ ]:
instance_list = list()
for q_ind, d_ind in tqdm.tqdm(relass.items()):
    temp_query = [queries[query_ids.index(q_ind)]]
    temp_documents = [documents[doc_ids.index(idx)] for idx in d_ind]
    temp_list = list(product(temp_query, temp_documents))  # list of tuples
    
    # if negative examples to be added, then for each training instance that has the same query, create a 
    # different set of negative examples, the number of negative examples is set at the beginning
    
    if num_negative is not None:
        d_ind_list = [doc_ids.index(idx) for idx in d_ind]  
        for q in temp_list:
            temp_negatives = find_negative(d_ind_list, num_negative)
            temp_tuple = (*q, *temp_negatives)
            instance_list.append(temp_tuple)
    else:
        instance_list.extend(temp_list)

assert len(instance_list) == len(ids)

 79%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                     | 33/42 [00:08<00:02,  3.50it/s]

In [ ]:
instance_dic = defaultdict(list)
for q, d, *negatives in instance_list:
    # For the naming convengtion, "article" is used, whereas article_1 and article_2 are reserved for query and document
    instance_dic["article_1"].append(q)
    instance_dic["article_2"].append(d)
    
    # If negatives to be added    
    if num_negative is not None:
        for idx, neg in enumerate(negatives, 3):
            instance_dic[f"article_{idx}"].append(neg)

# Create Dataset

In [ ]:
dataset = Dataset.from_dict(instance_dic)
print(dataset[0])

In [418]:
dataset = dataset.train_test_split(test_size=0.1)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['article_1', 'article_2', 'article_3', 'article_4', 'article_5', 'article_6', 'article_7', 'article_8', 'article_9', 'article_10'],
        num_rows: 738
    })
    test: Dataset({
        features: ['article_1', 'article_2', 'article_3', 'article_4', 'article_5', 'article_6', 'article_7', 'article_8', 'article_9', 'article_10'],
        num_rows: 83
    })
})


In [419]:
path = os.path.join(root_dir, f"clef_{year}_{num_negative}")
dataset.save_to_disk(path)